In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation, LeakyReLU
from keras import backend
from keras.callbacks import TensorBoard
from keras.optimizers import Adam
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
# Hyperparams
n_units = 400
layers = 4
n_batch = 1024
n_epochs = 200

In [3]:
df = pd.read_csv('options-df-sigma.csv')
df = df.dropna(axis=0)
df = df.drop(columns=['date', 'exdate', 'impl_volatility'])
df.strike_price = df.strike_price / 1000
call_df = df[df.cp_flag == 'C'].drop(['cp_flag'], axis=1)
put_df = df[df.cp_flag == 'P'].drop(['cp_flag'], axis=1)

In [4]:
call_df.head()

,strike_price,best_bid,best_offer,volume,open_interest,date_ndiff,treasury_rate,closing_price,sigma_20
0,600.0,28.000,29.000,4,8555,47,5.17,624.22,0.007761
10,475.0,152.875,153.875,0,2700,145,5.12,624.22,0.007761
13,600.0,52.375,53.375,0,2155,327,5.05,624.22,0.007761
17,610.0,20.000,20.750,923,8860,47,5.17,624.22,0.007761
18,675.0,34.000,35.000,0,350,691,5.10,624.22,0.007761


In [10]:
call_X_train, call_X_test, call_y_train, call_y_test = train_test_split(call_df.drop(['best_bid', 'best_offer'], axis=1),
                                                                        call_df[['best_bid', 'best_offer']],
                                                                        test_size=0.01, random_state=42)
put_X_train, put_X_test, put_y_train, put_y_test = train_test_split(put_df.drop(['best_bid', 'best_offer'], axis=1),
                                                                    put_df[['best_bid', 'best_offer']],
                                                                    test_size=0.01, random_state=42)

In [12]:
model = Sequential()
model.add(Dense(n_units, input_dim=call_X_train.shape[1]))
model.add(LeakyReLU())

for _ in range(layers - 1):
    model.add(Dense(n_units))
    model.add(LeakyReLU())

model.add(Dense(2, activation='relu'))

model.compile(loss='mse', optimizer=Adam(lr=0.0001))

Instructions for updating:
Colocations handled automatically by placer.


In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 400)               3200      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 400)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 400)               160400    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 400)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 400)               160400    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 400)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 400)               160400    
__________

In [52]:
history = model.fit(call_X_train, call_y_train, 
                    batch_size=n_batch, epochs=n_epochs, 
                    validation_split = 0.01,
                    callbacks=[TensorBoard()],
                    verbose=1)

Train on 5053632 samples, validate on 51047 samples
Epoch 1/200
5053632/5053632 [==============================] - 223s 44us/step - loss: 72933.1162 - val_loss: 72543.9143
Epoch 2/200
5053632/5053632 [==============================] - 216s 43us/step - loss: 25807.4910 - val_loss: 119.9612
Epoch 3/200
5053632/5053632 [==============================] - 217s 43us/step - loss: 120.8947 - val_loss: 102.9109
Epoch 4/200
 192512/5053632 [>.............................] - ETA: 3:34 - loss: 108.9445- ETA: 3:

KeyboardInterrupt: 